# 符合Python风格的对象

## 对象表示形式

Python 对象的字符串表示形式

- `repr()` 以便于开发者理解的方式返回对象的字符串表示形式
- `str()` 以便于用户理解的方式返回对象的字符串表示形式
- `bytes()` 函数调用`__bytes__`获取对象的字节序列表示形式
- `__format__` 方法会被内置的 `format()` 函数和 `str.format()` 方法调用


`__repr__` 和 `__str__` 特殊方法，为`repr()` 和 `str()` 提供支持

## 再谈向量类

实现Vector2d

In [ ]:
from array import array
import math

class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        """
        使对象可迭代，这样才能够拆包
        """
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

## `classmethod`与`staticmethod`

使用`@staticmethod`或`@classmethod`装饰一个类方法，就可以不需要实例化，直接类名.方法名()来调用

二者区别在于，
- `@staticmethod`不需要表示自身对象的self和自身类的cls参数，就跟使用函数一样
- `@classmethod`也不需要self参数，但第一个参数需要是表示自身类的cls参数

如果在`@staticmethod`中要调用到这个类的一些属性方法，只能直接类名.属性名或类名.方法名。
而`@classmethod`因为持有cls参数，可以来调用类的属性，类的方法，实例化对象等，避免硬编码。

In [1]:
class A(object):
    bar = 1
    def foo(self):
        print('foo')

    @staticmethod
    def static_foo():
        print('static_foo')
        print(A.bar)

    @classmethod
    def class_foo(cls):
        print('class_foo')
        print(cls.bar)
        cls().foo()
###执行
A.static_foo()
A.class_foo()

static_foo
1
class_foo
1
foo


## 格式化显示

内置的 `format()` 函数和 `str.format()` 方法把各个类型的格式化方式委托给相应的 `.__format__(format_spec)` 方法。

 format_spec 是格式说明符
 - `format(my_obj, format_spec)`的第二个参数
 - `str.format()`方法的格式化字符串，`{}`里代换字段中冒号后面的部分

各个类可以自行决定如何解释`format_spec` 参数，类没有定义 `__format__` 方法，
则会使用`__str__`方法，但不能使用格式说明符

In [3]:
# 为Vector2d自定义的类型扩展格式规范
def __format__(self, fmt_spec=''):
    components = (format(c, fmt_spec) for c in self)
    return '({},{})'.format(*components)

## 可散列的`Vector2d`

想把对象放入集合中，需要实现`__hash__`和`__eq__`，实例的散列值绝不应该变化

`__hash__`应返回一个整数值，且属性相同的对象(`__eq__`实现)的散列值值相同

## 私有属性和“受保护的”属性

以`__mood` 的形式（两个前导下划线，尾部没有或最多有一个下划线）命名实例属性，
Python 会把属性名存入实例的`__dict__` 属性中，而且会在前面加上一个下划线和类名。
这个语言特性叫**名称改写**

约定使用一个下划线前缀编写“受保护”的属性（如 self._x）

## 使用 `__slots__` 类属性节省空间

默认情况下， Python 在各个实例中名为 `__dict__` 的字典里存储实例属性。

通过`__slots__`类属性，能节省大量内存，方法是让解释器在元组中存储实例属性，而不用字典